<a href="https://colab.research.google.com/github/meti-94/OpenQA/blob/main/MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/meti-94/OpenQA.git

Cloning into 'OpenQA'...
remote: Enumerating objects: 215, done.
remote: Counting objects: 100% (215/215), done.
remote: Compressing objects: 100% (180/180), done.
remote: Total 215 (delta 85), reused 128 (delta 28), pack-reused 0
Receiving objects: 100% (215/215), 102.55 MiB | 31.09 MiB/s, done.
Resolving deltas: 100% (85/85), done.


In [3]:
import pandas as pd
import sys
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

In [4]:
df = pd.read_excel('/content/OpenQA/validation.xlsx')

In [5]:
df.loc[1, 'answers']

"[('20e3dd8b-0f8e-4379-8552-58aef7ca3857', 'again chose', 1.0, 1.0, '0.90925', 114750)]"

In [6]:
X, y = [], []
for idx1, row in df.iterrows():
  try:
    temporary = eval(row['answers'])
  
    for idx2, candidate in enumerate(temporary):
      if len(candidate)==6:
        if (candidate[0]==row['Answer']) and candidate[-1]==row['Reverb_no']:
          X.append((candidate[2], candidate[3], float(candidate[4])))
          y.append(1)
        else:
          X.append((candidate[2], candidate[3], float(candidate[4])))
          y.append(0)
      else:
        if (candidate[0]==row['Answer']) and (candidate[1]==row['Relation']):
          X.append((candidate[2], candidate[3], 1.0))
          y.append(1)
        else:
          X.append((candidate[2], candidate[3], 1.0))
          y.append(0)
  except:
    continue


In [7]:
len(X), len(y), sum(y), X[:4]

(223245,
 223245,
 16306,
 [(1.0, 1.0000000000000002, 0.92794),
  (1.0, 1.0, 0.90925),
  (1.0, 1.0, 0.9384100000000001),
  (1.0, 0.9467529114555084, 0.9338200000000001)])

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [9]:
regr = MLPRegressor(hidden_layer_sizes=(3, 10, 10, ),
                    activation='tanh',
                    solver='adam',
                    learning_rate='adaptive',
                    ).fit(X_train, y_train)

In [10]:
regr.predict(X_test[:2])

array([-0.00215009, -0.02727856])

In [11]:
regr.score(X_test, y_test)

0.25874595977517756

In [22]:
def prepare_model_input(candidate):
  try:
    if len(candidate)==6:
      return [candidate[2], candidate[3], float(candidate[4])]
    else:
      return [candidate[2], candidate[3], 1.0]
  except:
    return [0, 0, 0]
def sort_by_mlp_score(candidates):
  try:
    scored_candidates = sorted(eval(candidates), 
                              key = lambda item: regr.predict([prepare_model_input(item)]),
                              reverse=True)
    return scored_candidates
  except:
    return []

In [24]:
df.answers = df.answers.apply(sort_by_mlp_score)

In [26]:
data = df
top1 = 0
top3 = 0
top5 = 0
top10 = 0
top20 = 0
top50 = 0
top100 = 0
for idx1, row in data.iterrows():
  for idx2, candidate in enumerate(row['answers']):
    if len(candidate)==6:
      if (candidate[0]==row['Answer']) and candidate[-1]==row['Reverb_no']:
        if idx2 in range(1):
            # print(candidate, row['Reverb_no'])
            top1 += 1
        if idx2 in range(3):
            top3 += 1
        if idx2 in range(5):
            top5 += 1
        if idx2 in range(10):
            top10 += 1
        if idx2 in range(20):
            top20 += 1
        if idx2 in range(50):
            top50 += 1
        if idx2 in range(100):
            top100 += 1
            break 
    else:
      if (candidate[0]==row['Answer']) and (candidate[1]==row['Relation']):
        if idx2 in range(1):
            # print(candidate, row['Answer'], row['Relation'])
            top1 += 1
        if idx2 in range(3):
            top3 += 1
        if idx2 in range(5):
            top5 += 1
        if idx2 in range(10):
            top10 += 1
        if idx2 in range(20):
            top20 += 1
        if idx2 in range(50):
            top50 += 1
        if idx2 in range(100):
            top100 += 1
            break 
   

# print(data_type)
total = len(data)
print("Top1 Answers: {}".format(top1 / total))
print("Top3 Answers: {}".format(top3 / total))
print("Top5 Answers: {}".format(top5 / total))
print("Top10 Answers: {}".format(top10 / total))
print("Top20 Answers: {}".format(top20 / total))
print("Top50 Answers: {}".format(top50 / total))
print("Top100 Answers: {}".format(top100 / total))

Top1 Answers: 0.7181832643970728
Top3 Answers: 0.796293350302259
Top5 Answers: 0.8141902640789055
Top10 Answers: 0.8312917594654788
Top20 Answers: 0.8389277760101813
Top50 Answers: 0.8470410435889277
Top100 Answers: 0.8495068405981546


In [ ]:
import pickle
with open('classifier.pkl', 'wb') as fid:
    pickle.dump(regr, fid)    